In [1]:
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)

In [2]:
import os, sys, time

In [3]:
import ROOT

Welcome to JupyROOT 6.10/09


In [4]:
from importlib import import_module
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import PostProcessor

In [5]:
from PhysicsTools.NanoAODTools.postprocessing.framework.datamodel import Collection, Object
from PhysicsTools.NanoAODTools.postprocessing.framework.eventloop import Module

In [6]:
class MFaJaT(Module):
    def __init__(self):
        self.writeHistFile=True
    def beginJob(self,histFile=None,histDirName=None):
        Module.beginJob(self,histFile,histDirName)
        
        self.h_lumiblock = ROOT.TH1I('h_lumiblock', ':LumiBlock:Events',   100, 0, 1000)
        self.addObject(self.h_lumiblock)
        self.h_run = ROOT.TH1I('h_run', ':Run:Events', 10, 0, 10)
        self.addObject(self.h_run)
    def endJob(self):
        pass
    #def beginFile(self, inputFile, outputFile):
    #def beginFile(self):
        #pass
    #def endFile(self, inputFile, outputFile):
    #def endFile(self):
        #pass
    def analyze(self, event):
        """process event, return True (go to next module) or False (fail, go to next event)"""
        
        ###########################################
        ###### Basic Attributes of the Event ######
        ###########################################
        run = getattr(event, "run")
        lumi = getattr(event, "luminosityBlock")
        evt = getattr(event, "event")
        
        self.h_lumiblock.Fill(lumi)
        self.h_run.Fill(run)
        #print("Run: {0:>8d} LuminosityBlock: {2:>8d} Event: {1:>8d}".format(run, evt, lumi)) #swizzling because lazy
        
        
        return True #Exit early, to just print this info for a json file test...
        
        ################################
        ###### High Level Trigger ######
        ################################
        HLT = Object(event, "HLT") #Grab all the HLT triggers 'together'
        passTrig=["PFMETNoMu90_PFMHTNoMu90_IDTight"] #Create a list of valid triggers to check, dropping "HLT_"
        for trig in passTrig:
            print("HLT_" + str(trig) + " Trigger: " + str(getattr(HLT, trig)) )
            
            
        ###############################
        ######### MET Filters #########
        ###############################
        Filters = Object(event, "Flag") #Grab all the MET Filters in a group
        passFilter=["HBHENoiseFilter", "HBHENoiseIsoFilter", "EcalDeadCellTriggerPrimitiveFilter", 
                    "globalSuperTightHalo2016Filter", "goodVertices", "METFilters"] #All the filters commonly used 
        for fltr in passFilter:
            print("Flag_" + str(fltr) + " Filter: " + str(getattr(Filters, fltr)))
        
        
        ###########################################
        ###### Event Collections and Objects ######
        ###########################################
        electrons = Collection(event, "Electron")
        photons = Collection(event, "Photon")
        muons = Collection(event, "Muon")
        taus = Collection(event, "Tau")
        jets = Collection(event, "Jet")
        met = Object(event, "MET")
        PV = Object(event, "PV")
        SV = Collection(event, "SV")
        
        ###############################
        ###### Time To Do Stuff? ######
        ###############################
        eventSum = ROOT.TLorentzVector()
        print("PV  X: {0: >5.3f} Y: {1: >5.3f} Z: {2:5.3f} nDoF: {3: >5f} Chi^2: {4: >5.3f}".format(
            PV.x,PV.y, PV.z, PV.ndof, PV.chi2))
        if len(SV) > 0:   
            print("nSV: {0: >3d} SV[0] Decay Length:{1: >5.3f}".format(len(SV), SV[0].dlen ))
        else:
            print("nSV: {0: >3d}".format(len(SV)))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Muon", "Pt", "Eta", "Phi"))
        for nm, lep in enumerate(muons) :
            eventSum += lep.p4()
            #format_spec ::=  [[fill]align][sign][#][0][width][,][.precision][type]
            print("{0:*<5d} {1:>10.4f} {2:>+10.3f} {3:>+10.3f}".format(nm+1, lep.pt, lep.eta, lep.phi))
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Electron", "Pt", "Eta", "Phi"))
        for ne, lep in enumerate(electrons) :
            eventSum += lep.p4()
            print("{0:*^5d} {1:>10.4f} {2:> 10.3f} {3:> 10.3f}".format(ne+1, lep.pt, lep.eta, lep.phi))
        #for j in filter(self.jetSel,jets):
        print("{0:>5s} {1:>10s} {2:>10s} {3:>10s}".format("Jet", "Pt", "Eta", "Phi"))
        for nj, j in enumerate(jets):
            eventSum += j.p4()
            print("{0: >5d} {1:>10.4f} {2:>-10.3f} {3:>-10.3f}".format(nj+1, j.pt, j.eta, j.phi))
        print("Event Mass: {:<10.4f}\n".format(eventSum.M()))
        
        ###########################################
        ###### Return True to pass the event ######
        ###########################################
        return True


In [7]:
#preselection="nMuon > 0 && nElectron > 0"
preselection=None
filePrefix = "root://cms-xrd-global.cern.ch/"
inputList = open("../data/Run2016/Run2016B_SM", "r")
files=[]
for line in inputList:
    files.append(filePrefix + str(line))
#for file in files:
#    print(file)
onefile = [files[0]] #Experiment with just one file for now...
#onefile = " root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root"
print(onefile)

[u'root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root\n']


In [11]:
p=PostProcessor(".",onefile,cut=preselection,branchsel=None,modules=[MFaJaT()],friend=False,postfix="_MFaJaT", 
                jsonInput=None,noOut=True,justcount=False,provenance=False,
                haddFileName=None,fwkJobReport=False,histFileName="histOut.root",
                histDirName="plots", outputbranchsel=None)
#jsonInput: json file in dictionary format {"RunNumberInt": [[lumilow,lumihigh],[lumi2low,lumi2high]], "RunNumber2Int":[[low,high]]}
#files: list ["fileone.root","filetwo.root"] of inputs, even if one file must be a list!
#branchsel: if non-None, selection of branches to not even activate/load into memory
#outputbranchsel: if non-None, selection of branches to still include in output (see noOut)
#noOut: If True, no output of skimmed data is written. If False, will write full data file fitting outputbranch selections and postfix concattenated to name
#postfix: string added to inputfile name to indicate this module processed it!
#histFileName: name of any output file for histograms created in your class, as above with runs and lumis
#histDirName: name INSIDE the "histFileName.root" file's directory structure!
#friend: Not tested
#haddFileName: Not properly tested (tied together with fwkJobReport)
#fwkJobReport: only relevant with multiple files being added together? Not properly tested

SysError in <TFile::Flush>: error flushing file histOut.root (No such file or directory)


In [10]:
p.run()

Pre-select 11647 entries out of 11647 
Processed    10000/   11647 entries (elapsed time     1.0s, curr speed    9.804 kHz, avg speed    9.804 kHz), accepted    10001/   10001 events (100.00%)
Processed 11647 preselected entries from root://cms-xrd-global.cern.ch//store/data/Run2016B/SingleMuon/NANOAOD/05Feb2018_ver2-v1/00000/2055267F-3110-E811-8956-AC1F6B1AEF94.root
 (11647 entries). Finally selected 11647 entries
8501.45985401 Hz
